In [2]:
%cd /homes/kb658/yinyang/

/homes/kb658/yinyang


In [3]:
import yaml
import jsonlines

config_path = "/homes/kb658/yinyang/configs/configs_os.yaml"

# Load config file
with open(config_path, 'r') as f:
    configs = yaml.safe_load(f)
    
artifact_folder = configs["raw_data"]["artifact_folder"]
mono_folder = configs["raw_data"]["mono_folder"]
json_folder = configs["raw_data"]["json_folder"]
raw_data_folders = configs["raw_data"]["raw_data_folders"]

In [4]:
import glob

data_path = raw_data_folders['classical']['folder_path']

# Get all the midi files in the data path recursively
midi_files = glob.glob(data_path + '/**/*.midi', recursive=True)

In [5]:
len(midi_files)

1276

In [6]:
from aria.data.midi import MidiDict
from aria.tokenizer import AbsTokenizer

tokenizer = AbsTokenizer()
midi_file_path = "/import/c4dm-datasets/maestro-v3.0.0/2008/MIDI-Unprocessed_07_R2_2008_01-05_ORIG_MID--AUDIO_07_R2_2008_wav--2.midi" # midi_files[1]
print(midi_file_path)
mid = MidiDict.from_midi(midi_file_path)
tokenized_sequence = tokenizer.tokenize(mid)

/import/c4dm-datasets/maestro-v3.0.0/2008/MIDI-Unprocessed_07_R2_2008_01-05_ORIG_MID--AUDIO_07_R2_2008_wav--2.midi


In [8]:
# midi_file_path

# # Copy this file to the current directory
# import shutil
# import os

# shutil.copy(midi_file_path, 'test.midi')

In [7]:
len(tokenized_sequence)

1549

In [8]:
tokenized_sequence[0:150]

[('prefix', 'instrument', 'piano'),
 '<S>',
 ('piano', 63, 75),
 ('onset', 0),
 ('dur', 920),
 ('piano', 64, 75),
 ('onset', 860),
 ('dur', 740),
 ('piano', 63, 75),
 ('onset', 1320),
 ('dur', 280),
 ('piano', 68, 60),
 ('onset', 1470),
 ('dur', 820),
 ('piano', 71, 75),
 ('onset', 1480),
 ('dur', 830),
 ('piano', 62, 75),
 ('onset', 1490),
 ('dur', 790),
 ('piano', 70, 75),
 ('onset', 2180),
 ('dur', 1090),
 ('piano', 61, 60),
 ('onset', 2180),
 ('dur', 1090),
 ('piano', 67, 60),
 ('onset', 2180),
 ('dur', 1080),
 ('piano', 39, 45),
 ('onset', 2180),
 ('dur', 1080),
 ('piano', 68, 60),
 ('onset', 3040),
 ('dur', 3280),
 ('piano', 44, 45),
 ('onset', 3060),
 ('dur', 3260),
 ('piano', 51, 30),
 ('onset', 4100),
 ('dur', 2220),
 ('piano', 56, 30),
 ('onset', 4120),
 ('dur', 2200),
 ('piano', 59, 30),
 ('onset', 4130),
 ('dur', 2200),
 ('piano', 63, 30),
 ('onset', 4130),
 ('dur', 2180),
 '<T>',
 ('piano', 70, 45),
 ('onset', 460),
 ('dur', 860),
 ('piano', 71, 60),
 ('onset', 840),
 ('du

In [10]:
tokenized_sequence[-20:]

[('piano', 68, 60),
 ('onset', 4590),
 ('dur', 4340),
 ('piano', 44, 45),
 ('onset', 4600),
 ('dur', 4330),
 '<T>',
 ('piano', 63, 45),
 ('onset', 870),
 ('dur', 3060),
 ('piano', 59, 30),
 ('onset', 890),
 ('dur', 3040),
 ('piano', 51, 30),
 ('onset', 930),
 ('dur', 3000),
 ('piano', 56, 30),
 ('onset', 940),
 ('dur', 3000),
 '<E>']

In [15]:
# Define a function to flatten the tokenized sequence
def flatten(sequence):
    flattened_sequence = []
    note_info = []
    for i in range(len(sequence)):
        if sequence[i] == "<T>":
            flattened_sequence.append(sequence[i])
        if sequence[i][0] == "piano":
            note_info.append(sequence[i][1])
            note_info.append(sequence[i][2])
        elif sequence[i][0] == "onset":
            note_info.append(sequence[i][1])
        elif sequence[i][0] == "dur":
            note_info.append(sequence[i][1])
            flattened_sequence.append(note_info) 
            note_info = []

    return flattened_sequence

def skyline(sequence: list, diff_threshold=50):
    melody = []
    pitch = 0
    velocity = sequence[0][1]
    onset = sequence[0][2]
    duration = sequence[0][3]
    i = 0

    for i in range(len(sequence)):
        if type(sequence[i]) != str:
            current_pitch = sequence[i][0]
            current_velocity = sequence[i][1]
            current_onset = sequence[i][2]
            current_duration = sequence[i][3]

            # Get next note onset
            if i+2 < len(sequence):
                if type(sequence[i+1]) == str and type(sequence[i+2]) == str:
                    next_onset = 5000
                elif type(sequence[i+1]) == str and type(sequence[i+2]) != str:
                    next_onset = sequence[i+2][2]
                else:
                    next_onset = sequence[i+1][2]
            else:
                next_onset = sequence[-1][2]
            
            # Get difference between current and next onset
            diff_next_curr_onset = abs(next_onset - current_onset)

            # Get difference between current and previous onset
            diff_curr_prev_onset = abs(current_onset - onset)

            # Check if current_onset is different from next onset
            if diff_next_curr_onset > diff_threshold:
                # Check if current_onset is different from previous onset and if current_pitch is greater than pitch
                if diff_curr_prev_onset < diff_threshold and current_pitch > pitch:
                    pitch = current_pitch
                    onset = current_onset
                    duration = current_duration
                    velocity = current_velocity
                elif diff_curr_prev_onset >= diff_threshold:
                    pitch = current_pitch
                    onset = current_onset
                    duration = current_duration
                    velocity = current_velocity
                else:
                    continue
                # Append the last note
                melody.append(("piano", pitch, velocity))
                melody.append(("onset", onset))
                melody.append(("dur", duration))

            else:
                if current_pitch > pitch:
                    pitch = current_pitch
                    onset = current_onset
                    duration = current_duration
                    velocity = current_velocity
                else:
                    continue
        else:
            melody.append(sequence[i])
    return melody
                
# Call the flatten function
flattened_sequence = flatten(tokenized_sequence)
print(flattened_sequence[0:100])

# Call the skyline function
melody = skyline(flattened_sequence, diff_threshold=30)
melody = tokenized_sequence[0:2] + melody + tokenized_sequence[-1:]
print(melody[0:100], '\n')

[[63, 75, 0, 920], [64, 75, 860, 740], [63, 75, 1320, 280], [68, 60, 1470, 820], [71, 75, 1480, 830], [62, 75, 1490, 790], [70, 75, 2180, 1090], [61, 60, 2180, 1090], [67, 60, 2180, 1080], [39, 45, 2180, 1080], [68, 60, 3040, 3280], [44, 45, 3060, 3260], [51, 30, 4100, 2220], [56, 30, 4120, 2200], [59, 30, 4130, 2200], [63, 30, 4130, 2180], '<T>', [70, 45, 460, 860], [71, 60, 840, 480], [73, 75, 1110, 300], [75, 75, 1360, 620], [78, 75, 1840, 180], [76, 60, 1940, 780], [75, 60, 2570, 890], [80, 60, 2950, 510], [75, 60, 3470, 2070], [68, 45, 4300, 1240], [71, 45, 4330, 1210], [63, 30, 4340, 1200], [56, 30, 4340, 1200], [75, 75, 4920, 620], '<T>', [73, 75, 370, 180], [76, 75, 500, 720], [70, 60, 500, 140], [61, 45, 520, 700], [49, 60, 530, 700], [68, 45, 530, 690], [75, 75, 1130, 660], [68, 45, 1150, 640], [71, 45, 1160, 150], [56, 45, 1160, 630], [44, 45, 1170, 620], [73, 60, 1630, 160], [71, 60, 1740, 600], [70, 60, 2220, 220], [73, 60, 2390, 810], [68, 45, 2410, 790], [64, 45, 2410, 7

In [16]:
melody[-10:]

[('onset', 2440),
 ('dur', 480),
 ('piano', 39, 45),
 ('onset', 3620),
 ('dur', 1190),
 '<T>',
 ('piano', 59, 30),
 ('onset', 890),
 ('dur', 3040),
 '<E>']

In [19]:
print(tokenized_sequence[1:150])

['<S>', ('piano', 67, 90), ('onset', 0), ('dur', 570), ('piano', 65, 90), ('onset', 560), ('dur', 530), ('piano', 70, 90), ('onset', 1070), ('dur', 550), ('piano', 72, 90), ('onset', 1580), ('dur', 250), ('piano', 77, 90), ('onset', 1820), ('dur', 290), ('piano', 74, 90), ('onset', 2070), ('dur', 550), ('piano', 72, 90), ('onset', 2580), ('dur', 280), ('piano', 77, 90), ('onset', 2830), ('dur', 300), ('piano', 74, 90), ('onset', 3100), ('dur', 550), ('piano', 70, 90), ('onset', 3600), ('dur', 550), ('piano', 72, 90), ('onset', 4120), ('dur', 590), ('piano', 67, 90), ('onset', 4660), ('dur', 550), '<T>', ('piano', 65, 90), ('onset', 180), ('dur', 580), ('piano', 67, 90), ('onset', 730), ('dur', 500), ('piano', 55, 60), ('onset', 770), ('dur', 470), ('piano', 58, 60), ('onset', 770), ('dur', 470), ('piano', 43, 60), ('onset', 770), ('dur', 470), ('piano', 62, 45), ('onset', 770), ('dur', 460), ('piano', 65, 75), ('onset', 1240), ('dur', 530), ('piano', 45, 60), ('onset', 1260), ('dur', 5

In [17]:
print(len(tokenized_sequence))
print(len(melody))

1549
504


In [18]:
mid_dict = tokenizer.detokenize(melody) # mid_dict is a MidiDict object
mid = mid_dict.to_midi() # mid is a mido.MidiFile
mid.save('test_file.mid')